### Main code

In [ ]:
# from PIL import Image

# image = rle_to_mask(
#     rle_id,  # result['value']['rle']
#     original_height,  # result['original_height']
#     original_width # result['original_width']
# )

# print(image.shape)  # (original_height, original_width)

# Image.fromarray(image).show()

In [1]:
from typing import List

def cv2_rect_to_yolo(x1,x2,y1,y2, w_size, h_size):
    w=(x2-x1)/w_size
    h=(y2-y1)/h_size
    x = x1/w_size + w/2
    y = y1/h_size + h/2

    return x,y,w,h


def write_yolo(list_yolo_labels:List[List[int]], 
               path:str):
    """
    list_yolo_labels =[ [clas, x, y, w, h], [clas, x, y, w, h] ] = List[  List[int, int, int, int, int]  ]
    """
    str_data = ""
    for i in list_yolo_labels: # clas, x, y, w, h
        i=" ".join(map(str, i))
        str_data+=f"\n{i}"

    str_data=str_data.strip()+"\n"
    with open(path, "w") as text_file:
        text_file.write(str_data)
    print("file writed")

In [2]:
def mask2yolo(arr):
    
    # print(f"{arr.shape}")
    y_len, x_len = arr.shape

    # find upper bound = Y1
    for r in range(y_len):
        if arr[r:r+1,:][0].sum()!=0:
            upper_bound = r # y1
            break


    # find down bound = Y2
    for r in reversed(range(y_len)):
        # print(f"r={r}")
        # print(f"arr[r:r+1,:] {arr[r:r+1,:]}")
        if arr[r:r+1,:][0].sum()!=0:
            down_bound = r+1 # y2
            break

    # find left bound = X1
    for r in range(x_len):
        # print(f"r={r}")
        # print(f"arr[:,r:r+1].sum() {arr[:,r:r+1].sum()}")
        if arr[:,r:r+1].sum()!=0:
            left_bound = r # x1
            break


    # find right bound = X2
    for r in reversed(range(x_len)):
        if arr[:,r:r+1].sum()!=0:
            right_bound = r+1 # x2
            break
    x1x2y1y2 = (left_bound, right_bound, upper_bound, down_bound)
    return x1x2y1y2


In [3]:
def relative2absalute(np_arr, resolution_orig):
    np_arr_abs = np.copy(np_arr)
    np_arr_abs[:,:1] = np_arr_abs[:,:1]/100*resolution_orig[1]
    np_arr_abs[:,1:] = np_arr_abs[:,1:]/100*resolution_orig[0]
    # np_arr_abs=np.rint(np_arr_abs).astype(np.int32)
    np_arr_abs=np.rint(np_arr_abs).astype(np.int32)

    return np_arr_abs


def get_resolution(one_imgObj_LS):
    w_orig = one_imgObj_LS["annotations"][0]['result'][0]['original_width']
    h_orig = one_imgObj_LS["annotations"][0]['result'][0]['original_height']
    resolution_orig = (h_orig,w_orig)
    return resolution_orig

In [4]:
from PIL import Image
import numpy as np
import cv2 as cv

from IPython.display import display

class LSseg_2_yolo:
    def __init__(self, path_yolo_out, classes):
        self.classes = classes# {"leaf" :0, "tomato":1, "stem":2}
        self.path_yolo_out = path_yolo_out
        
    def run(self, item_label):
        original_width = item_label["annotations"][0]["result"][0]["original_width"]
        original_height = item_label["annotations"][0]["result"][0]["original_height"]

        # Base name
        image_name = item_label['data']['image']
        image_name = os.path.basename(image_name)
        base_name = image_name[:image_name.rfind(".")]
        # print(f"image_name {image_name}")

        # np_array_base = np.zeros([original_height, original_width], dtype=np.uint8)
        list_yolo_labels=[]

        resolution_orig=get_resolution(item_label)
        
        for result in item_label["annotations"][0]["result"]:
            # Labels classes
            label = result["value"]["polygonlabels"][0]
            cl_label = self.classes[label]
        
            points=result["value"]['points']
            np_polyg=np.array( [np.array(i) for i in points] )
            # Convert relative points to absolute
            np_polyg_abs=relative2absalute(np_polyg, resolution_orig)
            # Add dimension polyg
            np_polyg_abs=np_polyg_abs[np.newaxis, ... ] 
            # Create temporaty mask
            mask = np.zeros(shape=resolution_orig, dtype=np.uint8)

            image_np = cv.fillPoly(mask, pts=np_polyg_abs, color=254)
            # img_pill = Image.fromarray(filled_mask)
            # display(img_pill)
        
            if image_np.max()==0:
                continue
            image_np[image_np != 0] = 254 # cl_label / value_label

            img_seg=Image.fromarray(image_np)

            # # Get value for cropp
            x1,x2,y1,y2 = mask2yolo(image_np)

            # Converting abolute rectangle point [x1,x2,y1,y2] to => relative yolo [x, y, w, h]
            x,y,w,h = cv2_rect_to_yolo(x1,x2,y1,y2, original_width, original_height)
            # Writing results to yolo label file
            list_yolo_labels.append([cl_label, x, y, w, h])

        path_yolo_label = os.path.join(self.path_yolo_out, f"{base_name}.txt")
        write_yolo(list_yolo_labels, path_yolo_label)
                 

In [ ]:
# import json
# import os
# from tqdm import tqdm

# path_labeled = "/Users/YaVolkonskiy/Documents/001_Projects/003_tomato/001_data/001_raw_data/010_data_greenhouse_20_12_23/selected_images/Gelya/export_LS_seg/project-20-at-2024-02-01-17-10-4e3676dd.json"
# path_yolo_out = "/Users/YaVolkonskiy/Documents/001_Projects/003_tomato/001_data/001_raw_data/010_data_greenhouse_20_12_23/selected_images/Gelya/yolo_bbox"
# classes = {"leaf" :0, "tomato":1, "stem":2}

# with open(path_labeled) as user_file:
#     parsed_json = json.load(user_file)
# parsed_json

In [6]:
import json
import os
from tqdm import tqdm

path_labeled = "/Users/YaVolkonskiy/Documents/001_Projects/003_tomato/001_data/001_raw_data/002_video_data_customer/002_labeled/labeled_Olga_Gelya_21_11_23/project-13-at-2023-11-21-11-39-a223ddf0.json"
path_yolo_out = "/Users/YaVolkonskiy/Documents/001_Projects/003_tomato/001_data/001_raw_data/002_video_data_customer/002_labeled/labeled_Olga_Gelya_21_11_23/yolo_bbox/labels"
classes = {"leaf" :0, "tomato":1, "stem":2}

with open(path_labeled) as user_file:
    parsed_json = json.load(user_file)

for item_label in tqdm(parsed_json):
    c=LSseg_2_yolo(path_yolo_out, classes)
    image_np=c.run(item_label)


  1%|█▉                                                                                                                                                                                      | 1/94 [00:01<02:20,  1.51s/it]

file writed


  2%|███▉                                                                                                                                                                                    | 2/94 [00:03<02:30,  1.64s/it]

file writed


  3%|█████▊                                                                                                                                                                                  | 3/94 [00:04<02:02,  1.35s/it]

...

file writed


 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████  | 93/94 [02:40<00:01,  1.73s/it]

file writed


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 94/94 [02:42<00:00,  1.73s/it]

file writed


### Train test scripts

In [ ]:
image_np.max()

In [ ]:
arr = np.zeros([12,6])
arr[4:7,3:5]=1
arr

In [ ]:
# arr[11][-1]=100
# arr[11][-2]=99
# arr[10][-2]=98
# arr[0][0]=50


arr

In [ ]:
arr

In [ ]:


# mask2yolo(arr)    

In [ ]:
x1,x2,y1,y2 =mask2yolo(arr) 
print(f"x1,x2,y1,y2 {x1,x2,y1,y2}")
arr[y1:y2, x1:x2]

In [ ]:
r=1
arr[:,r:r+1][0]

In [ ]:
arr[:,4:11]

In [ ]:
image_np[8:9,:]

In [ ]:
arr

In [ ]:
a=[0,1,2,3,4]

" ".join(map(str, a))

In [ ]:
a = "image_name 148.png"

a[:a.rfind(".")]